## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zharkent,KZ,44.1628,80.0000,79.32,31,10,4.00,clear sky
1,1,Hobart,AU,-42.8794,147.3294,57.92,96,75,11.50,light intensity drizzle
2,2,Nanortalik,GL,60.1432,-45.2371,36.91,92,100,7.94,overcast clouds
3,3,Montivilliers,FR,49.5452,0.1877,60.60,94,90,11.50,overcast clouds
4,4,Pisco,PE,-13.7000,-76.2167,62.65,82,90,4.61,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Zharkent,KZ,44.1628,80.0000,79.32,31,10,4.00,clear sky
6,6,Rikitea,PF,-23.1203,-134.9692,76.06,86,99,14.81,overcast clouds
7,7,Sur,OM,22.5667,59.5289,93.76,53,6,4.14,clear sky
13,13,Axim,GH,4.8699,-2.2405,78.37,86,95,4.07,light rain
14,14,Atuona,PF,-9.8000,-139.0333,79.61,74,32,15.61,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Zharkent,KZ,79.32,clear sky,44.1628,80.0000,
6,Rikitea,PF,76.06,overcast clouds,-23.1203,-134.9692,
7,Sur,OM,93.76,clear sky,22.5667,59.5289,
13,Axim,GH,78.37,light rain,4.8699,-2.2405,
14,Atuona,PF,79.61,scattered clouds,-9.8000,-139.0333,
16,Souillac,MU,77.36,overcast clouds,-20.5167,57.5167,
20,Cockburn Town,TC,80.13,scattered clouds,21.4612,-71.1419,
22,Adrar,MR,87.53,scattered clouds,20.5022,-10.0711,
30,Hithadhoo,MV,83.17,scattered clouds,-0.6000,73.0833,
33,Jurado,CO,77.07,moderate rain,7.1053,-77.7641,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
# Replace empty hotel names with NaN
hotel_df.replace("",float("NaN"), inplace=True)
# Drop the rows with NaN values
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Zharkent,KZ,79.32,clear sky,44.1628,80.0000,гостиница Satti
6,Rikitea,PF,76.06,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Sur,OM,93.76,clear sky,22.5667,59.5289,Sur Plaza Hotel
13,Axim,GH,78.37,light rain,4.8699,-2.2405,AXIM BEACH RESORT AND SPA
14,Atuona,PF,79.61,scattered clouds,-9.8000,-139.0333,Villa Enata
16,Souillac,MU,77.36,overcast clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
20,Cockburn Town,TC,80.13,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
30,Hithadhoo,MV,83.17,scattered clouds,-0.6000,73.0833,Scoop Guest House
34,Vanimo,PG,84.40,few clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
37,Cabo San Lucas,MX,82.40,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))